In [ ]:
##load data
import os
import pickle
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import time
import gc
print('Loading data...', flush=True)
dataset_path = '/data/home/mrichte3/DTIAM/code/5-20-24/'
thread_num = 64
os.environ['OMP_NUM_THREADS'] = '64'
batch_size = 128
comp_features_train = []
training_length = 45000000
test_length = training_length // 10
for i in range(17):
    file_name = os.path.join(dataset_path, f'train_fold_smiles.csvcomp_feat_{i}.pkl')
    with open(file_name, 'rb') as file:
        comp_feat = pickle.load(file)
        comp_features_train.extend(comp_feat)
    print(f"File {i}: {file_name}, Number of entries: {len(comp_feat)}")
    if len(comp_features_train) >= training_length:
        break
    gc.collect()
comp_features_train = comp_features_train[:training_length]
train_data = np.array(comp_features_train, dtype=np.float32)
train_labels = pd.read_csv(os.path.join(dataset_path, 'train_fold_0.csv'))['label'].values[:training_length]
pos_weight = len(train_labels) / (train_labels == 1).sum()
print(pos_weight)
train_data_tensor = torch.tensor(train_data, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
train_dataset = torch.utils.data.TensorDataset(train_data_tensor, train_labels_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=thread_num)
del comp_features_train, train_data, train_data_tensor, train_labels_tensor
gc.collect()
file_name = os.path.join(dataset_path, 'test_fold_smiles.csvcomp_feat_0.pkl')
with open(file_name, 'rb') as file:
    comp_features_test = pickle.load(file)
test_data = np.array(comp_features_test, dtype=np.float32)[:test_length]
test_labels = pd.read_csv(os.path.join(dataset_path, 'test_fold_0.csv'))['label'].values[:test_length]
test_data_tensor = torch.tensor(test_data, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)
test_dataset = torch.utils.data.TensorDataset(test_data_tensor, test_labels_tensor)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=thread_num)
del comp_features_test, test_data, test_data_tensor, test_labels_tensor
gc.collect()
print('Data loaded. Starting model...', flush=True)


In [ ]:
import importlib
import distributed_training10  # Import the module first

importlib.reload(distributed_training10)
from distributed_training10 import main

main(train_loader, test_loader, pos_weight)
#main(train_loader, test_loader, thread_num)
#main(train_loader, test_loader)